In [1]:
using Pkg
cd("D:\\Home\\Git\\Oceananigans.jl")
# cd("C:\\Users\\Ali\\Documents\\Git\\Oceananigans.jl\\")
Pkg.activate(".");

In [2]:
using Statistics, Printf, BenchmarkTools
using Oceananigans, Oceananigans.Operators

In [3]:
N = (100, 100, 100)
L = (2000, 2000, 2000)

c = EarthConstants()
eos = LinearEquationOfState()

g = RegularCartesianGrid(N, L; dim=3, FloatType=Float64)

U  = VelocityFields(g)
tr = TracerFields(g)
pr = PressureFields(g)
G  = SourceTerms(g)
Gp = SourceTerms(g)
F  = ForcingFields(g)
tmp = TemporaryFields(g)

U.u.data  .= 0
U.v.data  .= 0
U.w.data  .= 0
tr.S.data .= 35
tr.T.data .= 283;

In [4]:
function δρ1!(s::LinearEquationOfState, g::Grid, δρ::CellField, tr::TracerFields)
    for k in 1:g.Nz, j in 1:g.Ny, i in 1:g.Nx
        @inbounds δρ.data[i, j, k] =  s.ρ₀ * s.βT * (tr.T.data[i, j, k] - s.T₀)
    end
    nothing
end

δρ1! (generic function with 1 method)

In [5]:
function δρ2!(s::LinearEquationOfState, g::Grid, δρ::CellField, T::CellField)
    for k in 1:g.Nz, j in 1:g.Ny, i in 1:g.Nx
        @inbounds δρ.data[i, j, k] =  s.ρ₀ * s.βT * (T.data[i, j, k] - s.T₀)
    end
    nothing
end

δρ2! (generic function with 1 method)

In [6]:
@benchmark δρ1!($eos, $g, $tmp.fC1, $tr)

BenchmarkTools.Trial: 
  memory estimate:  91.55 MiB
  allocs estimate:  6000001
  --------------
  minimum time:     167.566 ms (1.22% GC)
  median time:      175.389 ms (1.26% GC)
  mean time:        177.779 ms (1.29% GC)
  maximum time:     209.425 ms (1.15% GC)
  --------------
  samples:          29
  evals/sample:     1

In [7]:
@benchmark δρ2!($eos, $g, $tmp.fC1, $tr.T)

BenchmarkTools.Trial: 
  memory estimate:  80 bytes
  allocs estimate:  1
  --------------
  minimum time:     644.346 μs (0.00% GC)
  median time:      739.161 μs (0.00% GC)
  mean time:        794.915 μs (0.00% GC)
  maximum time:     1.811 ms (0.00% GC)
  --------------
  samples:          6240
  evals/sample:     1

In [22]:
function ∫dz1(g::Grid, c::PlanetaryConstants, δρ::CellField, δρz::FaceFieldZ, pr::PressureFields)
    for j in 1:g.Ny, i in 1:g.Nx
      pr.pHY′.data[i, j, 1] = δρ.data[i, j, 1] * c.g * g.Δz / 2
    end
    for k in 2:g.Nz, j in 1:g.Ny, i in 1:g.Nx
      pr.pHY′.data[i, j, k] = pr.pHY′.data[i, j, k-1] + (δρz.data[i, j, k] * c.g * g.Δz)
    end
end

∫dz1 (generic function with 2 methods)

In [17]:
function ∫dz2(g::Grid, c::PlanetaryConstants, δρ::CellField, δρz::FaceFieldZ, pHY′::CellField)
    for j in 1:g.Ny, i in 1:g.Nx
      pHY′.data[i, j, 1] = δρ.data[i, j, 1] * c.g * g.Δz / 2
    end
    for k in 2:g.Nz, j in 1:g.Ny, i in 1:g.Nx
      pHY′.data[i, j, k] = pHY′.data[i, j, k-1] + (δρz.data[i, j, k] * c.g * g.Δz)
    end
end

∫dz2 (generic function with 2 methods)

In [18]:
δρ = tmp.fC1
δρ!(eos, g, δρ, tr.T)
@. tr.ρ.data = eos.ρ₀ + δρ.data

δρz = tmp.fFZ
avgz!(g, δρ, δρz)

In [23]:
@benchmark ∫dz1($g, $c, $δρ, $δρz, $pr)

BenchmarkTools.Trial: 
  memory estimate:  45.47 MiB
  allocs estimate:  2980000
  --------------
  minimum time:     124.506 ms (0.96% GC)
  median time:      129.043 ms (0.92% GC)
  mean time:        130.316 ms (0.96% GC)
  maximum time:     149.386 ms (0.80% GC)
  --------------
  samples:          39
  evals/sample:     1

In [24]:
@benchmark ∫dz2($g, $c, $δρ, $δρz, $pr.pHY′)

BenchmarkTools.Trial: 
  memory estimate:  32 bytes
  allocs estimate:  1
  --------------
  minimum time:     3.588 ms (0.00% GC)
  median time:      3.966 ms (0.00% GC)
  mean time:        4.131 ms (0.00% GC)
  maximum time:     7.567 ms (0.00% GC)
  --------------
  samples:          1208
  evals/sample:     1